In [2]:
import json
from ipywidgets import interactive, Button, HBox, VBox, IntSlider
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# ---unique--------------------------------
theta1_deg_values = np.linspace(1.2, 2.6, num=8)
theta2_deg_values = np.linspace(0.6, 1.6, num=6)
A_values = np.linspace(0.5e-3, 3.0e-3, num=26)

# Function to create an interactive plot
def interactive_plot_func(mf, mr, l, I, V):
    l = l / 1000  # (mm -> m)
    V = V / 3.6  # (kph -> m/s)
    a = 4.0  # [m/s^2]
    m = mf + mr
    
    df_all = pd.DataFrame()
    df_all['A_values'] = A_values*1e3
    
    fig, ax = plt.subplots()

    x_max = 2.2
    y_max = 3.0
    ax.set_xlim(0.8, x_max)
    ax.set_ylim(0.5, y_max)
    ax.set_xlabel("Yaw resonance $\omega_n$ [Hz]")
    ax.set_ylabel("A [$10^{-3} s^2/m^2$]")
    ax.grid()

    for t1_deg in theta1_deg_values:
        t1 = t1_deg / 180 * np.pi
        omega_n = []
        for a_val in A_values:
            t2 = t1 - a * a_val * l
            K_f = (mf * a) / (2 * t1)
            K_r = (mr * a) / (2 * t2)
            omega = ((2 * l / V) * np.sqrt(K_f * K_r / (m * I)) * np.sqrt(1 + a_val * V ** 2))/(2*np.pi)
            omega_n.append(omega)
        ax.plot(omega_n, A_values*1e3, label=f"theta1={t1_deg:.1f}deg", color='blue')
        df_all[f"t1={round(t1_deg, 1)}"] = omega_n
        # Setting the legend text position to the point located at the top right within the frame of the graph
        within_limits = [(x, y) for x, y in zip(omega_n, A_values*1e3) if x <= x_max and y <= y_max]
        text_pos = max(within_limits, default=(x_max, y_max), key=lambda item: item[0] + item[1])    
        ax.text(text_pos[0], text_pos[1], f"{t1_deg:.1f}", ha='left', va='center', color='blue')

    for t2_deg in theta2_deg_values:
        t2 = t2_deg / 180 * np.pi
        omega_n = []
        for a_val in A_values:
            t1 = t2 + a * a_val * l
            K_f = (mf * a) / (2 * t1)
            K_r = (mr * a) / (2 * t2)
            omega = ((2 * l / V) * np.sqrt(K_f * K_r / (m * I)) * np.sqrt(1 + a_val * V ** 2))/(2*np.pi)
            omega_n.append(omega)
        ax.plot(omega_n, A_values*1e3, label=f"theta2={t2_deg:.1f}deg", color='red')
        df_all[f"t2={round(t2_deg, 1)}"] = omega_n
        # Setting the legend text position to the point located at the top right within the frame of the graph
        within_limits = [(x, y) for x, y in zip(omega_n, A_values*1e3) if x <= x_max and y <= y_max]
        text_pos = max(within_limits, default=(x_max, y_max), key=lambda item: item[0] + item[1])    
        ax.text(text_pos[0], text_pos[1], f"{t2_deg:.1f}", ha='left', va='center', color='red')

    plt.show()
    return df_all

def save_data(button):
    df_all = interactive_plot_func(**{child.description.split(' ')[0]: child.value for child in interactive_plot.children[:-1]})
    df_all.to_csv('data.csv', index=False)
    
# ---partially unique--------------------------------

# define a dictionary that links the description to the name and unit
slider_parameters = {
    "mf": {"name": "Front mass", "unit": "kg"},
    "mr": {"name": "Rear mass", "unit": "kg"},
    "l": {"name": "Wheelbase", "unit": "mm"},
    "I": {"name": "Yaw inertia", "unit": "kgm^2"},
    "V": {"name": "Velocity", "unit": "kph"},
}

# Creating the sliders
spec_sliders = {
    "mf": IntSlider(min=500, max=2000, step=10, value=1000, description=f'mf [{slider_parameters["mf"]["unit"]}]', continuous_update=False),
    "mr": IntSlider(min=500, max=2000, step=10, value=700, description=f'mr [{slider_parameters["mr"]["unit"]}]', continuous_update=False),
    "l": IntSlider(min=2000, max=4000, step=10, value=2700, description=f'l [{slider_parameters["l"]["unit"]}]', continuous_update=False),
    "I": IntSlider(min=2000, max=4000, step=10, value=2650, description=f'I [{slider_parameters["I"]["unit"]}]', continuous_update=False)
}
condition_sliders = {
    "V": IntSlider(min=30, max=120, step=30, value=120, description=f'V [{slider_parameters["V"]["unit"]}]', continuous_update=False)
}

# ---common--------------------------------

interactive_plot = interactive(interactive_plot_func,
                               **spec_sliders, 
                               **condition_sliders)

# Function to define the operation of the Save button
def save_values(b):
    spec_slider_values = {
        key: {
            'name': slider_parameters[key]['name'],
            'value': slider.value,
            'unit': slider_parameters[key]['unit']
        }
        for key, slider in spec_sliders.items()
    }
    try:
        with open('spec_defaults.json', 'r+') as f:
            data = json.load(f)
            data.update(spec_slider_values)
            f.seek(0)
            f.write(json.dumps(data))
            f.truncate()
    except FileNotFoundError:
        with open('spec_defaults.json', 'w') as f:
            json.dump(spec_slider_values, f)
            
# Function to define the operation of the Load button
def load_values(b):
    try:
        with open('spec_defaults.json', 'r') as f:
            defaults = json.load(f)
            for key, slider in spec_sliders.items():
                if key in defaults:
                    slider.value = defaults[key]["value"]
    except FileNotFoundError:
        print("No default values found. Please save first.")

# create buttons
save_param_button = Button(description="Save parameters")
save_param_button.on_click(save_values)

load_param_button = Button(description="Load parameters")
load_param_button.on_click(load_values)

save_data_button = Button(description="Save data")
save_data_button.on_click(save_data)

# Displaying the UI elements
display(HBox([interactive_plot, VBox([save_param_button, load_param_button, save_data_button])]))